In [53]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [54]:
url = '/content/drive/MyDrive/AlmaBetter /Capstone Project/Book Recommendation System/Books_data.csv'
books_url = '/content/drive/MyDrive/AlmaBetter /Capstone Project/Book Recommendation System/Book RCD Dataset /Copy of Books.csv'

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import seaborn as sns

In [57]:
df = pd.read_csv(url , dtype = {'ISBN': str, 'Rating': int, 'Uid': str} )
books = pd.read_csv(books_url)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [58]:
df.head()

,Unnamed: 0,Uid,ISBN,Rating,Title,Author,Pub_Year,Publisher,Image_Url,Location,Age,age_segment
0,3649,638,0446520802,0,the notebook,nicholas sparks,1996,warner books,http://images.amazon.com/images/p/0446520802.0...,"san diego, california, usa",20.0,Adult
1,3650,638,0316666343,10,the lovely bones: a novel,alice sebold,2002,"little, brown",http://images.amazon.com/images/p/0316666343.0...,"san diego, california, usa",20.0,Adult
2,3652,638,0385504209,10,the da vinci code,dan brown,2003,doubleday,http://images.amazon.com/images/p/0385504209.0...,"san diego, california, usa",20.0,Adult
3,3653,638,0345452550,0,a cold heart: an alex delaware novel,jonathan kellerman,2003,ballantine books,http://images.amazon.com/images/p/0345452550.0...,"san diego, california, usa",20.0,Adult
4,3654,638,0679746048,7,"girl, interrupted",susanna kaysen,1994,vintage,http://images.amazon.com/images/p/0679746048.0...,"san diego, california, usa",20.0,Adult


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252588 entries, 0 to 252587
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   252588 non-null  int64  
 1   Uid          252588 non-null  object 
 2   ISBN         252588 non-null  object 
 3   Rating       252588 non-null  int64  
 4   Title        252588 non-null  object 
 5   Author       252588 non-null  object 
 6   Pub_Year     252588 non-null  int64  
 7   Publisher    252588 non-null  object 
 8   Image_Url    252588 non-null  object 
 9   Location     252588 non-null  object 
 10  Age          252588 non-null  float64
 11  age_segment  252588 non-null  object 
dtypes: float64(1), int64(3), object(8)
memory usage: 23.1+ MB


## Creating Memory Based Models

In [60]:
# drop columns which are not really required
df.drop(['Image_Url', 'Location','Publisher','Unnamed: 0' ],axis = 1, inplace = True)

In [61]:
df.head()

,Uid,ISBN,Rating,Title,Author,Pub_Year,Age,age_segment
0,638,0446520802,0,the notebook,nicholas sparks,1996,20.0,Adult
1,638,0316666343,10,the lovely bones: a novel,alice sebold,2002,20.0,Adult
2,638,0385504209,10,the da vinci code,dan brown,2003,20.0,Adult
3,638,0345452550,0,a cold heart: an alex delaware novel,jonathan kellerman,2003,20.0,Adult
4,638,0679746048,7,"girl, interrupted",susanna kaysen,1994,20.0,Adult


In [62]:
df.shape

(252588, 8)

# **KNN based Recommendation system**


In [63]:
# Create a pivot table
df_pivot = df.pivot_table(index = 'Title',columns = 'Uid', values = 'Rating',).fillna(0)
df_pivot.head()

Uid,100004,100009,10001,100053,100066,100088,100115,100119,100121,100227,...,99523,99616,99630,99648,99738,99765,99792,99955,99996,99997
Title,,,,,,,,,,,,,,,,,,,,,
"q-space (star trek the next generation, book 47)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'salem's lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01-01-00: the novel of the millennium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"1,000 places to see before you die",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 lb. penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
# create a sparse matrix from df_pivot
from scipy.sparse import csr_matrix

df_matrix = csr_matrix(df_pivot.values)

In [65]:
# create a Nearest neighbors model with cosine similarity  
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [66]:
df_pivot.shape

(8852, 7888)

In [67]:
#select a random book index and then find its distance and indices
query_index = np.random.choice(df_pivot.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(df_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

4261


In [68]:
# function to find out the top 5 similar books
def top_5_books(query_index):
  distances, indices = model_knn.kneighbors(df_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
  for i in range(0, len(distances.flatten())):
      if i == 0:
          print('Recommendations for {0}:\n'.format(df_pivot.index[query_index]))
      else:
          print('{0}: {1}, with distance of {2}:'.format(i, df_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

In [69]:
# get a name of book from the user and recommend him 5 books using top_5_books function
book = input('enter the name of book you like: ')
try:
  n = df_pivot.index == book
  query_index = int(np.where(n == True)[0])
  top_5_books(query_index)
except:
  print(f'try for another book, Book not Found')

enter the name of book you like: the notebook
Recommendations for the notebook:

1: a walk to remember, with distance of 0.786688296529096:
2: the rescue, with distance of 0.8047108577109342:
3: message in a bottle, with distance of 0.8098782120586079:
4: prozac nation: young and depressed in america : a memoir, with distance of 0.8116455130548362:
5: sybil, with distance of 0.8304540677623126:


# **Correlation based Recommendation system**

In [70]:
df_pivot.head()

Uid,100004,100009,10001,100053,100066,100088,100115,100119,100121,100227,...,99523,99616,99630,99648,99738,99765,99792,99955,99996,99997
Title,,,,,,,,,,,,,,,,,,,,,
"q-space (star trek the next generation, book 47)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'salem's lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01-01-00: the novel of the millennium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"1,000 places to see before you die",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 lb. penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
# Transpose the matrix
ndf_pivot = df_pivot.T

In [72]:
def corr_recomendation(book):
  cor_vector = ndf_pivot.corrwith( ndf_pivot[book], )
  cor_vector.sort_values(ascending= False, inplace= True)
  print(f"top movies similar to {cor_vector.iloc[:1].index } are")
  print(f'{cor_vector.iloc[2:7] }', end = '\n')

In [73]:
book = input('enter the name of book you like: ')
try:
  corr_recomendation(book)
except:
  print('your book is not in list, Please Retry....')

enter the name of book you like: the notebook
top movies similar to Index(['the notebook'], dtype='object', name='Title') are
Title
the rescue                                                  0.185776
prozac nation: young and depressed in america : a memoir    0.184958
message in a bottle                                         0.179124
sybil                                                       0.167220
the wedding                                                 0.161602
dtype: float64


### Issues
extremely Time consuming memory consuming

# Model Based recommendation engine
using collaborative filtering

---
Latent filter SVD model

In [74]:
# df.drop('Unnamed: 0', axis = 1, inplace = True)

In [75]:
# rating df, to work on the useful features
rating_df = df[['Uid',	'ISBN',	'Rating',	'Title']].reset_index().drop('index', axis = 1)

In [76]:
rating_df.head()

,Uid,ISBN,Rating,Title
0,638,0446520802,0,the notebook
1,638,0316666343,10,the lovely bones: a novel
2,638,0385504209,10,the da vinci code
3,638,0345452550,0,a cold heart: an alex delaware novel
4,638,0679746048,7,"girl, interrupted"


In [77]:
rating_df['Uid'].value_counts()

153662    1483
76352     1455
16795     1122
198711    1113
204864    1007
          ... 
230050       5
239569       5
132166       5
14538        5
32444        5
Name: Uid, Length: 7888, dtype: int64

## Stratified Train test Split 

In [78]:
 # train test split
rating_train_df, rating_test_df = train_test_split(rating_df, 
                                                   test_size = 0.2, 
                                                   random_state = 10,
                                                   stratify= rating_df['Uid'])
print(f"train shape {rating_train_df.shape}")
print(f"test shape {rating_test_df.shape}")

train shape (202070, 4)
test shape (50518, 4)


In [79]:
# create a pivot of train set
ratings_train_pivot_df = rating_train_df.pivot_table(values = 'Rating', columns = 'ISBN', index = 'Uid').fillna(0)

In [80]:
# keep a list of user id's for an instace to use them further
user_ids = list(ratings_train_pivot_df.index) 
# keep values of ratings train df as a matrix
ratings_train_values = ratings_train_pivot_df.values

In [81]:
ratings_train_pivot_df.head(1)

ISBN,0002251760,0002558122,0006546684,0006547834,0006550576,0006550789,0007106572,0007110928,0007122039,0007141076,...,8806163698,8807810212,8807813025,8817106100,8817131628,8845205118,8845247414,8845915611,8885989403,9681500830
Uid,,,,,,,,,,,,,,,,,,,,,
100004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Svd Model to create factor matrix of original matrix

In [82]:
# The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(ratings_train_values, k = NUMBER_OF_FACTORS_MF)

In [83]:
# see the shapes of original matrix and all other matrix
print(f' shape of original matrix: {ratings_train_pivot_df.shape}')
print(f' shape of U factor matrix: {U.shape}')
print(f' shape of V-transpose matrix: {Vt.shape}')
sigma = np.diag(sigma) # To Create a diagnol matrix of the sigma
print(f' shape of sigma matrix: {sigma.shape}') 

 shape of original matrix: (7888, 9905)
 shape of U factor matrix: (7888, 15)
 shape of V-transpose matrix: (15, 9905)
 shape of sigma matrix: (15, 15)


## Predict Rating for all users

In [84]:
# multiply all small matrix to create a predicted final matrix for user ratings 
predicted_rating_for_users = np.dot(np.dot(U,sigma),Vt) 
# predicted rating matrix shape 
predicted_rating_for_users.shape 

(7888, 9905)

In [85]:
#Converting the reconstructed matrix back to a Pandas dataframe  
user_ids = [str(s) for s in user_ids]
col = [str(c) for c in  ratings_train_pivot_df.columns]
pred_df = pd.DataFrame(predicted_rating_for_users, columns = col, index=user_ids).transpose()  
pred_df.head()   

,100004,100009,10001,100053,100066,100088,100115,100119,100121,100227,...,99523,99616,99630,99648,99738,99765,99792,99955,99996,99997
0002251760,0.013315,0.037189,-0.000286,0.021554,0.005685,0.049519,-0.008331,0.001179,0.002317,0.100974,...,-0.000732,0.004524,-0.004371,0.004487,0.010235,0.000884,0.000854,0.003838,0.011200,-0.001053
0002558122,0.000457,0.000427,-0.000029,0.001008,0.000776,-0.001291,-0.000561,-0.000019,0.000015,-0.001596,...,-0.000397,0.000528,0.000152,-0.000035,0.000523,-0.000028,0.000254,0.000376,0.000182,-0.000019
0006546684,0.000072,0.000481,0.000011,0.000039,0.000049,0.000156,0.001137,0.000006,-0.000019,-0.000319,...,0.000521,-0.000093,0.000277,0.000201,0.000448,0.000052,0.000006,0.000403,0.000477,0.000256
0006547834,0.000345,0.029124,0.000056,0.001227,0.002579,0.030124,0.009901,0.000085,-0.000202,-0.006687,...,0.001942,0.000041,0.003021,0.001199,0.003742,0.000104,0.000723,0.003267,0.004050,0.001534
0006550576,0.005143,0.045086,-0.000067,0.003311,0.001310,0.068015,0.004129,0.000345,-0.000189,-0.002472,...,0.000477,-0.000622,0.001411,0.001357,0.002462,0.000267,0.000312,0.000061,0.003822,-0.000400


In [86]:
# try:
#   books.set_index('ISBN', inplace = True)
# except:
#   pass
# books.reset_index( inplace = True)

In [91]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, predictions_df, items_df=None):
        self.cf_predictions_df = predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False).reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating content that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions.index.isin(items_to_ignore)].sort_values('recStrength', ascending = False).head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            print(recommendations_df.head())
            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'index', 
                                                          right_on = 'ISBN')[['recStrength', 'ISBN', 'Book-Title', 
                                                                              'Year-Of-Publication','Publisher',
                                                                              'Book-Author']]


        return recommendations_df
    
cf_recommender_model = CFRecommender(pred_df, books)

In [92]:
cf_recommender_model.recommend_items( '388', verbose = True)

        index  recStrength
0  0316601950     0.092131
1  0312278586     0.088752
2  0312195516     0.085348
3  0446310786     0.083967
4  0060976845     0.083786


KeyError: ignored

In [96]:
# class for recommendation model 
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, predictions_df, items_df=None):
        self.predictions_df = predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items_by_user(self, user_id, items_to_ignore=[], topn=10, verbose=True):
        # Get and sort the user's predictions
        sorted_user_predictions = self.predictions_df[user_id].sort_values(ascending=False).reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating content that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions.index.isin(items_to_ignore)].sort_values('recStrength', ascending = False).head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            print(recommendations_df.head())
            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'index', 
                                                          right_on = 'ISBN')[['recStrength', 'ISBN', 'Book-Title', 
                                                                              'Year-Of-Publication','Publisher',
                                                                              'Book-Author']]

    


        return recommendations_df

    def recommend_items_by_item(self, book_id,topn=10, verbose=True):
      from sklearn.metrics.pairwise import cosine_similarity
      pdt = self.predictions_df.transpose()
      cosine_similarities = cosine_similarity(np.array(pdt[book_id]).reshape(1,-1),pred_df )
      recommended_df = pd.DataFrame( {'recStrength': cosine_similarities[0] }, index = pred_df.index ).sort_values(by = 'recStrength',ascending = False)
      recommended_df = recommended_df.iloc[:topn,]
      # print(recommended_df)
      if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            # print(recommended_df)
            recommended_df = recommended_df.merge(self.items_df, how = 'left', 
                                                          # left_on = 'index', 
                                                          left_index = True,
                                                          right_on = 'ISBN')[['recStrength', 'ISBN', 'Book-Title', 
                                                                              'Year-Of-Publication','Publisher',
                                                                              'Book-Author']]
      return recommended_df
    
cf_recommender_model = CFRecommender(pred_df, books)




In [101]:
# function to get all readen books by a user
def get_readen_books(u_id):
  read_notread = ratings_train_pivot_df.loc[u_id ,]
  readen = list(read_notread.iloc[read_notread.values!= 0].index) 
  return readen
get_readen_books

<function __main__.get_readen_books(u_id)>

In [105]:
cf_recommender_model.recommend_items_by_item('0060169281') 

,recStrength,ISBN,Book-Title,Year-Of-Publication,Publisher,Book-Author
8356,1.000000,0060169281,Soul Mates: Honoring the Mysteries of Love and...,1994,Harpercollins,Thomas Moore
9194,0.996561,0385302320,Voyager,1993,Delacorte Press,DIANA GABALDON
10716,0.992311,0345369432,California Gold,1990,Ballantine Books,John Jakes
9510,0.991163,3548359698,Warum MÃ?Â¤nner nicht zuhÃ?Â¶ren und Frauen sc...,2000,Ullstein Tb,Allan Pease
12619,0.990756,8845205118,Il Piccolo Principe Prince Italn,0,Bompiani Italian,Antoine de Saint-ExupÃ©ry
725,0.990587,2253005274,Le Grand Meaulnes (Classiques De Poche),2000,Distribooks Inc,Alain-Fournier
3035,0.990565,0525945172,The Experiment,1999,Penguin USA,John Darnton
15782,0.990125,2266022504,Si c'est un homme,1988,Presses Pocket,Primo Levi
13083,0.989634,0871138018,The Toughest Indian in the World,2000,Pub Group West,Sherman Alexie
7135,0.987458,0553205803,Even Cowgirls Get the Blues,1981,Bantam Books,Thomas Robbins


In [102]:
get_readen_books('114')

['0446608653', '0446612618', '0451208080', '0553584383', '0671027360']

### The Top-N accuracy metric choosen was **Recall@N** which evaluates whether the interacted item is among the top N items (hit) in the ranked list of 101 recommendations for a user.